# **Learn OpenUSD: An Introduction to Strength Ordering**

Welcome to the Jupyter notebook for *Learn OpenUSD: An Introduction to Strength Ordering*. This is where we will find all the Python activities related to this course. Before starting **Activity 1**, make sure to run the cell below.

>**NOTE**: Before starting make sure to run the cell below. This will install the relevant OpenUSD libraries that will be used through this notebook.

In [1]:
from utils.visualization import DisplayUSD
from utils.helperfunctions import create_new_stage

---

## **Activity 1**: Defining a Reference Prim

[References](https://openusd.org/release/glossary.html#usdglossary-references) are a [composition arc](https://openusd.org/release/glossary.html#usdglossary-compositionarcs). [References](https://openusd.org/release/glossary.html#usdglossary-references) are like links to separate pieces of a project, allowing you to include and reuse these pieces without copying them.

Here's an example of how a reference looks in `.usda`:

```python
#usda 1.0

def Xform "World"
{
    def Xform "Geometry"
    {
        def Xform "Box" (
            prepend references = @box/cubebox_a02_distilled/cubebox_a02_distilled.usd@
        )
        {
        }
    }
}
```

Firstly, we want to grab all references of a prim. To do this we use [`GetReferences()`](https://openusd.org/release/api/class_usd_prim.html#ac9081d27e9d2a1058e32249fb96aaa34). This returns a [`UsdReferences`](https://openusd.org/release/api/class_usd_references.html) object, which allows us to add, remove, and modify references.

To add a reference, we use [`AddReference()`](https://openusd.org/release/api/class_usd_references.html#a95bf456b23a234d3aa017015a4ad05e0). Let's see these in practice.

**Add the following code to the cell below, then run the cell:**

```python
# Define a reference prim and set its translation:
reference_prim = UsdGeom.Xform.Define(stage, world.GetPath().AppendPath("Cube_Ref")).GetPrim()
UsdGeom.XformCommonAPI(reference_prim).SetTranslate(Gf.Vec3d(5, 0, 0))

# Add a reference to the "cube.usda" file:
reference_prim.GetReferences().AddReference("./cube.usda")
```

In [2]:
from pxr import Usd, UsdGeom, Gf

# Create a new stage and define a cube:
file_path = "assets/cube.usda"
stage = create_new_stage(file_path)
cube = UsdGeom.Cube.Define(stage, "/Cube")
stage.SetDefaultPrim(cube.GetPrim())
stage.Save()

# Create a second file path and stage, define a world and a sphere:
second_file_path = "assets/sphere.usda"
stage = create_new_stage(second_file_path)
world = UsdGeom.Xform.Define(stage, "/World")
UsdGeom.Sphere.Define(stage, world.GetPath().AppendPath("Sphere"))

# Define a reference prim and set its translation:
reference_prim = UsdGeom.Xform.Define(stage, world.GetPath().AppendPath("Cube_Ref")).GetPrim()
UsdGeom.XformCommonAPI(reference_prim).SetTranslate(Gf.Vec3d(5, 0, 0))

# Add a reference to the "cube.usda" file:
reference_prim.GetReferences().AddReference("./cube.usda")


stage.Save()
DisplayUSD("assets/sphere.usda", show_usd_code=True)

Notice that the cube prim **DOES NOT** show up in the scene. 

This is due to a local opinion in our current layer stack. In the next part we will go over how we can adjust this using an xform to wrap around the cube.

---

## **Activity 2**: Understanding How LIVRPS Affects References

In the example above, you noticed that the cube prim defined in `cube.usda` was not showing up after we referenced it in `sphere.usda`. As we mentioned before, [references](https://openusd.org/release/glossary.html#usdglossary-references) are a [composition arc](https://openusd.org/release/glossary.html#usdglossary-compositionarcs).

In the current layer stack, we have a local opinion that ranks higher than our reference. `Xform` in `sphere.usda` is our local opinion which out weighs our reference `Cube` in `cube.usda`. 

One way to solve this is wrapping the cube prim in an `xform`. Below is the code change we would have to make.

**Add the following code to the cell below, then run the cell:**

``` python
# Define the world Xform:
world = UsdGeom.Xform.Define(stage, "/World")

# Define a cube and set it as the default prim:
cube = UsdGeom.Cube.Define(stage, world.GetPath().AppendPath("Cube"))
stage.SetDefaultPrim(world.GetPrim())
```

In [3]:
from pxr import Usd, UsdGeom, Gf

# Create a new stage 
file_path = "assets/cube.usda"
stage = create_new_stage(file_path)

# Define the world Xform:
world = UsdGeom.Xform.Define(stage, "/World")

# Define a cube and set it as the default prim:
cube = UsdGeom.Cube.Define(stage, world.GetPath().AppendPath("Cube"))
stage.SetDefaultPrim(world.GetPrim())


# Save the stage
stage.Save()


# Create a new stage and define the world xform
second_file_path = "assets/sphere.usda"
stage = create_new_stage(second_file_path)
world = UsdGeom.Xform.Define(stage, "/World")

# Define a sphere and set its parent to the world xform
UsdGeom.Sphere.Define(stage, world.GetPath().AppendPath("Sphere"))

# Define a reference prim and set its translation
reference_prim = UsdGeom.Xform.Define(stage, world.GetPath().AppendPath("Cube_Ref")).GetPrim()
UsdGeom.XformCommonAPI(reference_prim).SetTranslate(Gf.Vec3d(5, 0, 0))

# Add a reference to the :cube.usda: file
reference_prim.GetReferences().AddReference("./cube.usda")

# Save the stage
stage.Save()
DisplayUSD("assets/sphere.usda", show_usd_code=True)

---

## **Activity 3**: Adding References and Adding Attributes

[References](https://openusd.org/release/glossary.html#usdglossary-reference) in OpenUSD allow for the inclusion of external assets or sub-scene data into a scene. This mechanism helps in modularizing and reusing assets across different scenes and projects, enabling efficient management of large-scale 3D environments.

Here, we'll add a reference as an example for creating attributes in USD.

**Add the following code to the cell below, then run the cell:**

```python
# Define a child Xform named "Geometry" under the "World" Xform
geometry_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, world_xform.GetPath().AppendPath("Geometry"))

# Define a new Xform named "Box" under the root "Geometry" Xform
box_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, geometry_xform.GetPath().AppendPath("Box"))
box_prim: Usd.Prim = box_xform.GetPrim()

# Add a reference to a USD file containing a box geometry
box_prim.GetReferences().AddReference("box/cubebox_a02_distilled/cubebox_a02_distilled.usd")
```

In [4]:
from pxr import Usd, UsdGeom

file_path = "assets/custom_attributes.usda"
stage: Usd.Stage = create_new_stage(file_path)

# Define a root Xform named "World"
world_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, "/World")

# Define a child Xform named "Geometry" under the "World" Xform
geometry_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, world_xform.GetPath().AppendPath("Geometry"))

# Define a new Xform named "Box" under the root "Geometry" Xform
box_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, geometry_xform.GetPath().AppendPath("Box"))
box_prim: Usd.Prim = box_xform.GetPrim()

# Add a reference to a USD file containing a box geometry
box_prim.GetReferences().AddReference("box/cubebox_a02_distilled/cubebox_a02_distilled.usd")


# Get the property names of the box primitive
box_prop_names = box_prim.GetPropertyNames()

# Print each property name
for prop_name in box_prop_names:
    print(prop_name)

stage.Save()
DisplayUSD("assets/custom_attributes.usda", show_usd_code=True)

dimension
material:binding
physics:angularVelocity
physics:velocity
proxyPrim
purpose
semantic:QWQC:params:semanticData
semantic:QWQC:params:semanticType
semantic:QWQL:params:semanticData
semantic:QWQL:params:semanticType
semantic:QWQQ:params:semanticData
semantic:QWQQ:params:semanticType
semantic:Semantics_fK46:params:semanticData
semantic:Semantics_fK46:params:semanticType
visibility
xformOp:rotateXYZ
xformOp:scale
xformOp:translate
xformOpOrder



The [`UsdGeomXformOp`](https://openusd.org/release/api/class_usd_geom_xform_op.html) is a schema wrapper for `UsdAttribute`. 

We will see in the next cell how we can create attributes using `CreateAttribute()`.

Generally, we always should check if the attribute exists first before. The schema methods are clearer and easier to use.

**Add the following code to the cell below, then run the cell:**

```python
# Get the children of the box prim
box_prim_children = box_prim.GetChildren()
box_inst_children = box_prim_children[0].GetChildren()

# Create a Xform object from the second child of the box prim
box_xform: UsdGeom.Xform = UsdGeom.Xform(box_inst_children[2].GetChildren()[0])

# Add/Get Scale Op
if box_xform.GetScaleOp():
    box_scale_attr = box_xform.GetScaleOp()
else:
    box_scale_attr = box_xform.AddScaleOp()

# Set Scale Op
box_scale_attr.Set((2.0, 1.0, 1.0))
```

In [5]:
from pxr import Usd, UsdGeom

file_path = "assets/custom_attributes.usda"
stage: Usd.Stage = create_new_stage(file_path)


world_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, "/World")
geometry_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, world_xform.GetPath().AppendPath("Geometry"))

box_xform: UsdGeom.Xform = UsdGeom.Xform.Define(stage, geometry_xform.GetPath().AppendPath("Box"))
box_prim: Usd.Prim = box_xform.GetPrim()
box_prim.GetReferences().AddReference("box/cubebox_a02_distilled/cubebox_a02_distilled.usd")

# Get the children of the box prim
box_prim_children = box_prim.GetChildren()
box_inst_children = box_prim_children[0].GetChildren()

# Create a Xform object from the second child of the box prim
box_xform: UsdGeom.Xform = UsdGeom.Xform(box_inst_children[2].GetChildren()[0])

# Add/Get Scale Op
if box_xform.GetScaleOp():
    box_scale_attr = box_xform.GetScaleOp()
else:
    box_scale_attr = box_xform.AddScaleOp()

# Set Scale Op
box_scale_attr.Set((2.0, 1.0, 1.0))


stage.Save()
DisplayUSD("assets/custom_attributes.usda", show_usd_code=True)